Notebook used to convert ERA5 grib dataset to zarr files

In [ ]:
import os
import xarray as xr

: 

In [ ]:
data_dir = '../data/weather'
raw_filename = 'data.grib'

In [ ]:
# split for surface/sea type using grib command line tool
! grib_copy -w typeOfLevel=meanSea ../data/weather/data.grib ../data/weather/mwd_mwp_swh.grib
! grib_copy -w typeOfLevel=surface ../data/weather/data.grib ../data/weather/u10_v10.grib

In [ ]:
fp_u10_v10 = os.path.join(data_dir, 'u10_v10.grib')
u10_v10 = xr.open_dataset(fp_u10_v10, engine='cfgrib', decode_timedelta=True)

In [ ]:
u10_v10

In [ ]:
u10_v10.sizes

In [ ]:
new_coords = {
  'row' : [i for i in range(u10_v10.sizes.get('longitude'))],
  'col': [i for i in range(u10_v10.sizes.get('latitude'))]
}

new_coords

In [ ]:
u10_v10 = u10_v10.assign_coords(new_coords)

In [ ]:
u10 = u10_v10.drop_vars(['v10'])
v10 = u10_v10.drop_vars(['u10'])

In [ ]:
fp_u10 = os.path.join(data_dir, 'u10.zarr')
fp_v10 = os.path.join(data_dir, 'v10.zarr')

u10.to_zarr(fp_u10, mode='w')
v10.to_zarr(fp_v10, mode='w')

In [ ]:
fp_mwd_mwp_swh = os.path.join(data_dir, 'mwd_mwp_swh.grib')
mwd_mwp_swh = xr.open_dataset(fp_mwd_mwp_swh, engine='cfgrib', decode_timedelta=True)

In [ ]:
new_coords = {
  'row' : [i for i in range(mwd_mwp_swh.sizes.get('longitude'))],
  'col': [i for i in range(mwd_mwp_swh.sizes.get('latitude'))]
}

new_coords

In [ ]:
mwd_mwp_swh = mwd_mwp_swh.assign_coords(new_coords)

In [ ]:
mwd_mwp_swh

In [ ]:
variables = ["mwd", "mwp", "swh"]

for var in variables:
    ds = mwd_mwp_swh.drop_vars([v for v in variables if v != var])
    fp = os.path.join(data_dir, f"{var}.zarr")
    ds.to_zarr(fp, mode="w")
